In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../')

import fewshot.data
import fewshot.trainer
import fewshot.focal

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Refactoring tests on small dataset
We've refactored our data pipeline, utilites and training loop in the `fewshot` module. Let's test it below for a full run of training.

In [3]:
fewshot.data.fix_quotes('../data/fashion-dataset/styles.csv', '../data/fashion-dataset/styles_quoted.csv')

In [4]:
train_transform = transforms.Compose([
        transforms.Resize(300),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

In [8]:
data = fewshot.data.FashionData('../data/small/styles_quoted.csv',
                                '../data/small/images/',
                               train_transform=train_transform,
                               test_transform=test_transform,
                               top20=True)

In [9]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features

## replacing the last layer with the dense layer we need
model.fc = nn.Linear(num_ftrs, data.n_classes)

# Which loss performs better on top20?
### Plain Cross-Entropy:

In [11]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
fewshot.trainer.run_training(model, 20, data, optimizer, loss_func, gpu_id=0, root='./traces/')

------------- epoch:  0
Training loss: 0.732
Validation loss: 1.064
Validation accuracy: 0.737
------------- epoch:  1
Training loss: 0.450
Validation loss: 0.975
Validation accuracy: 0.801
------------- epoch:  2
Training loss: 0.386
Validation loss: 0.976
Validation accuracy: 0.795
------------- epoch:  3
Training loss: 0.349
Validation loss: 1.057
Validation accuracy: 0.816
------------- epoch:  4
Training loss: 0.320
Validation loss: 0.821
Validation accuracy: 0.817
------------- epoch:  5
Training loss: 0.281
Validation loss: 1.153
Validation accuracy: 0.792
------------- epoch:  6
Training loss: 0.284
Validation loss: 0.853
Validation accuracy: 0.840
------------- epoch:  7
Training loss: 0.259
Validation loss: 0.959
Validation accuracy: 0.837
------------- epoch:  8
Training loss: 0.241
Validation loss: 0.900
Validation accuracy: 0.842
------------- epoch:  9
Training loss: 0.243
Validation loss: 0.930
Validation accuracy: 0.832
------------- epoch:  10
Training loss: 0.225
Vali

In [15]:
best_checkpoint = torch.load('traces/gpu_0_Jun09_0014/model_epoch_18.pth')
model.load_state_dict(best_checkpoint['model_state_dict'])
c1, c5, occ = fewshot.trainer.evaluate(model.cuda(), data, 0, data.n_classes)
fewshot.trainer.pretty_print_eval(c1, c5, data)

Category                    Top-1     Top-5
-------------------------------------------
Tshirts                      89.1      99.2
Shirts                       98.3     100.0
Casual Shoes                 86.6      98.3
Watches                     100.0     100.0
Sports Shoes                 73.9     100.0
Kurtas                       94.9      99.1
Tops                         66.7     100.0
Handbags                     95.0      99.2
Heels                        89.7      99.1
Sunglasses                  100.0     100.0
Wallets                      95.7      99.1
Flip Flops                   87.6      96.5
Sandals                      78.4     100.0
Briefs                       95.5     100.0
Belts                        97.3      99.1
Backpacks                    84.2      99.1
Socks                        77.3      93.8
Formal Shoes                 77.8      99.1
Perfume and Body Mist         0.0       0.0
Jeans                        99.1     100.0
Average                      84.

### Focal Loss 

In [ ]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, data.n_classes)

loss_func = fewshot.focal.FocalLoss(gamma=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)
fewshot.trainer.run_training(model, 20, data, optimizer, loss_func, gpu_id=0, root='./traces/')

------------- epoch:  0


../fewshot/focal.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


Training loss: 0.463
Validation loss: 0.766
Validation accuracy: 0.690
------------- epoch:  1
Training loss: 0.274
Validation loss: 0.628
Validation accuracy: 0.772
------------- epoch:  2
Training loss: 0.209
Validation loss: 0.604
Validation accuracy: 0.779
------------- epoch:  3
Training loss: 0.173
Validation loss: 0.640
Validation accuracy: 0.788
------------- epoch:  4
Training loss: 0.159
Validation loss: 0.620
Validation accuracy: 0.774
------------- epoch:  5
Training loss: 0.155
Validation loss: 0.636
Validation accuracy: 0.791
------------- epoch:  6
Training loss: 0.133
Validation loss: 0.528
Validation accuracy: 0.831
------------- epoch:  7
Training loss: 0.133
Validation loss: 0.625
Validation accuracy: 0.810
------------- epoch:  8
Training loss: 0.117
Validation loss: 0.631
Validation accuracy: 0.814
------------- epoch:  9
Training loss: 0.127
Validation loss: 0.581
Validation accuracy: 0.833
------------- epoch:  10
Training loss: 0.106
Validation loss: 0.588
Valid

In [6]:
best_checkpoint = torch.load('traces/gpu_0_Jun08_1016/model_epoch_14.pth')
model.load_state_dict(best_checkpoint['model_state_dict'])
c1, c5, occ = fewshot.trainer.evaluate(model.cuda(), data, 0, data.n_classes)
fewshot.trainer.pretty_print_eval(c1, c5, data)

Category                    Top-1     Top-5
-------------------------------------------
Tshirts                      94.1     100.0
Shirts                       99.2     100.0
Casual Shoes                 91.6      98.3
Watches                     100.0     100.0
Sports Shoes                 66.4      98.3
Kurtas                       96.6     100.0
Tops                         66.7     100.0
Handbags                     92.4      99.2
Heels                        84.6      97.4
Sunglasses                  100.0     100.0
Wallets                      87.2     100.0
Flip Flops                   87.6      96.5
Sandals                      80.2     100.0
Briefs                       96.6     100.0
Belts                        99.1      99.1
Backpacks                    94.7     100.0
Socks                        80.4      96.9
Formal Shoes                 74.1      99.1
Perfume and Body Mist         0.0       0.0
Jeans                        99.1     100.0
Average                      84.

### Weighed Cross Entropy

In [5]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, data.n_classes)

loss_func = nn.CrossEntropyLoss(weight=torch.tensor(data.weights).float().cuda())
optimizer = optim.Adam(model.parameters(), lr=0.001)
fewshot.trainer.run_training(model, 15, data, optimizer, loss_func, gpu_id=0, root='./traces/')

------------- epoch:  0
Training loss: 0.875
Validation loss: 0.992
Validation accuracy: 0.637
------------- epoch:  1
Training loss: 0.544
Validation loss: 0.707
Validation accuracy: 0.698
------------- epoch:  2
Training loss: 0.463
Validation loss: 0.504
Validation accuracy: 0.767
------------- epoch:  3
Training loss: 0.370
Validation loss: 0.488
Validation accuracy: 0.783
------------- epoch:  4
Training loss: 0.345
Validation loss: 0.482
Validation accuracy: 0.799
------------- epoch:  5
Training loss: 0.335
Validation loss: 0.489
Validation accuracy: 0.785
------------- epoch:  6
Training loss: 0.300
Validation loss: 0.479
Validation accuracy: 0.816
------------- epoch:  7
Training loss: 0.289
Validation loss: 0.407
Validation accuracy: 0.837
------------- epoch:  8
Training loss: 0.278
Validation loss: 0.427
Validation accuracy: 0.801
------------- epoch:  9
Training loss: 0.262
Validation loss: 0.507
Validation accuracy: 0.808
------------- epoch:  10
Training loss: 0.264
Vali

In [7]:
best_checkpoint = torch.load('traces/gpu_0_Jun08_1445//model_epoch_14.pth')
model.load_state_dict(best_checkpoint['model_state_dict'])
c1, c5, occ = fewshot.trainer.evaluate(model.cuda(), data, 0, data.n_classes)
fewshot.trainer.pretty_print_eval(c1, c5, data)

Category                    Top-1     Top-5
-------------------------------------------
Tshirts                      85.7     100.0
Shirts                      100.0     100.0
Casual Shoes                 79.0      98.3
Watches                     100.0     100.0
Sports Shoes                 78.2     100.0
Kurtas                       86.3      99.1
Tops                         72.6     100.0
Handbags                     95.8      97.5
Heels                        85.5      99.1
Sunglasses                  100.0     100.0
Wallets                      94.0      99.1
Flip Flops                   90.3      97.3
Sandals                      80.2      99.1
Briefs                       95.5     100.0
Belts                       100.0      98.2
Backpacks                    93.0     100.0
Socks                        86.6      93.8
Formal Shoes                 97.2     100.0
Perfume and Body Mist         0.0       0.0
Jeans                        99.1     100.0
Average                      85.

### Weighed Focal Loss

In [6]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, data.n_classes)

normalized_weights = data.n_classes *data.weights /sum(data.weights)

loss_func = fewshot.focal.FocalLoss(gamma=.5, alpha=torch.tensor(normalized_weights).float().cuda())
optimizer = optim.Adam(model.parameters(), lr=0.001)
fewshot.trainer.run_training(model, 20, data, optimizer, loss_func, gpu_id=0, root='./traces/')

------------- epoch:  0


../fewshot/focal.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


Training loss: 0.027
Validation loss: 0.037
Validation accuracy: 0.620
------------- epoch:  1
Training loss: 0.018
Validation loss: 0.022
Validation accuracy: 0.719
------------- epoch:  2
Training loss: 0.013
Validation loss: 0.021
Validation accuracy: 0.757
------------- epoch:  3
Training loss: 0.012
Validation loss: 0.022
Validation accuracy: 0.728
------------- epoch:  4
Training loss: 0.011
Validation loss: 0.014
Validation accuracy: 0.783
------------- epoch:  5
Training loss: 0.010
Validation loss: 0.011
Validation accuracy: 0.824
------------- epoch:  6
Training loss: 0.009
Validation loss: 0.011
Validation accuracy: 0.821
------------- epoch:  7
Training loss: 0.009
Validation loss: 0.014
Validation accuracy: 0.794
------------- epoch:  8
Training loss: 0.008
Validation loss: 0.012
Validation accuracy: 0.829
------------- epoch:  9
Training loss: 0.008
Validation loss: 0.015
Validation accuracy: 0.808
------------- epoch:  10
Training loss: 0.008
Validation loss: 0.012
Valid

In [11]:
best_checkpoint = torch.load('traces/gpu_0_Jun08_1520/model_epoch_14.pth')
model.load_state_dict(best_checkpoint['model_state_dict'])
c1, c5, occ = fewshot.trainer.evaluate(model.cuda(), data, 0, data.n_classes)
fewshot.trainer.pretty_print_eval(c1, c5, data)

Category                    Top-1     Top-5
-------------------------------------------
Tshirts                      89.9     100.0
Shirts                       98.3     100.0
Casual Shoes                 80.7      97.5
Watches                      98.0     100.0
Sports Shoes                 79.8     100.0
Kurtas                       97.4     100.0
Tops                         61.5      96.6
Handbags                     90.8      96.6
Heels                        81.2      96.6
Sunglasses                  100.0     100.0
Wallets                      98.3     100.0
Flip Flops                   85.8      99.1
Sandals                      76.7     100.0
Briefs                       94.4     100.0
Belts                        98.2      99.1
Backpacks                    96.5     100.0
Socks                        93.8      97.9
Formal Shoes                 97.2     100.0
Perfume and Body Mist         0.0       0.0
Jeans                        99.1      99.1
Average                      85.

## Finetuning on the rare classes

In [13]:
data = fewshot.data.FashionData('../data/small/styles_quoted.csv',
                                '../data/small/images/',
                               train_transform=train_transform,
                               test_transform=test_transform,
                               top20=False)

In [15]:
best_checkpoint = torch.load('traces/gpu_0_Jun08_1520/model_epoch_14.pth')
model.load_state_dict(best_checkpoint['model_state_dict'])
num_ftrs = model.fc.in_features
## replacing the last layer with the dense layer we need
model.fc = nn.Linear(num_ftrs, data.n_classes)

In [16]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
fewshot.trainer.run_training(model, 15, data, optimizer, loss_func, gpu_id=0, root='./traces/')

------------- epoch:  0
Training loss: 1.942
Validation loss: 5.685
Validation accuracy: 0.318
------------- epoch:  1
Training loss: 1.100
Validation loss: 6.191
Validation accuracy: 0.245
------------- epoch:  2
Training loss: 0.941
Validation loss: 5.419
Validation accuracy: 0.375
------------- epoch:  3
Training loss: 0.828
Validation loss: 5.926
Validation accuracy: 0.380
------------- epoch:  4
Training loss: 0.730
Validation loss: 6.212
Validation accuracy: 0.397
------------- epoch:  5
Training loss: 0.707
Validation loss: 6.316
Validation accuracy: 0.393
------------- epoch:  6
Training loss: 0.633
Validation loss: 5.438
Validation accuracy: 0.416
------------- epoch:  7
Training loss: 0.615
Validation loss: 5.987
Validation accuracy: 0.409
------------- epoch:  8
Training loss: 0.555
Validation loss: 5.831
Validation accuracy: 0.416
------------- epoch:  9
Training loss: 0.532
Validation loss: 6.640
Validation accuracy: 0.379
------------- epoch:  10
Training loss: 0.511
Vali

In [21]:
best_checkpoint = torch.load('traces/gpu_0_Jun08_1557/model_epoch_12.pth')
model.load_state_dict(best_checkpoint['model_state_dict'])
c1, c5, occ = fewshot.trainer.evaluate(model.cuda(), data, 0, data.n_classes)
fewshot.trainer.pretty_print_eval(c1, c5, data)

Category                    Top-1     Top-5
-------------------------------------------
Shorts                       83.7      95.3
Trousers                     83.7      95.3
Flats                        85.7     100.0
Bra                         100.0     100.0
Dresses                      64.9      83.8
Sarees                        nan       nan
Earrings                    100.0     100.0
Deodorant                     0.0       0.0
Nail Polish                   0.0       0.0
Lipstick                      0.0       0.0
Track Pants                  74.4      97.7
Clutches                     78.8      93.9
Sweatshirts                  46.5      97.7
Caps                         87.5      95.0
Sweaters                     55.8      86.0
Ties                         91.2      94.1
Jackets                      86.0      97.7
Innerwear Vests               0.0       0.0
Kurtis                       71.4     100.0
Tunics                        7.7      92.3
Nightdress                    7.

## With weights

In [25]:
model.fc = nn.Linear(num_ftrs, 20)
best_checkpoint = torch.load('traces/gpu_0_Jun08_1520/model_epoch_14.pth')
model.load_state_dict(best_checkpoint['model_state_dict'])
num_ftrs = model.fc.in_features
## replacing the last layer with the dense layer we need
model.fc = nn.Linear(num_ftrs, data.n_classes)

loss_func = nn.CrossEntropyLoss(weight=torch.tensor(data.weights).float().cuda())
optimizer = optim.Adam(model.parameters(), lr=0.001)
fewshot.trainer.run_training(model, 20, data, optimizer, loss_func, gpu_id=0, root='./traces/')

------------- epoch:  0
Training loss: 4.105
Validation loss: 3.881
Validation accuracy: 0.112
------------- epoch:  1
Training loss: 2.938
Validation loss: 2.724
Validation accuracy: 0.266
------------- epoch:  2
Training loss: 2.264
Validation loss: 2.752
Validation accuracy: 0.249
------------- epoch:  3
Training loss: 1.971
Validation loss: 2.597
Validation accuracy: 0.304
------------- epoch:  4
Training loss: 1.759
Validation loss: 2.483
Validation accuracy: 0.293
------------- epoch:  5
Training loss: 1.607
Validation loss: 2.270
Validation accuracy: 0.346
------------- epoch:  6
Training loss: 1.355
Validation loss: 2.489
Validation accuracy: 0.332
------------- epoch:  7
Training loss: 1.285
Validation loss: 2.237
Validation accuracy: 0.328
------------- epoch:  8
Training loss: 1.158
Validation loss: 2.540
Validation accuracy: 0.353
------------- epoch:  9
Training loss: 1.058
Validation loss: 2.576
Validation accuracy: 0.349
------------- epoch:  10
Training loss: 1.032
Vali

In [24]:
best_checkpoint = torch.load('traces/gpu_0_Jun08_1617/model_epoch_1.pth')
model.load_state_dict(best_checkpoint['model_state_dict'])
c1, c5, occ = fewshot.trainer.evaluate(model.cuda(), data, 0, data.n_classes)
fewshot.trainer.pretty_print_eval(c1, c5, data)

Category                    Top-1     Top-5
-------------------------------------------
Shorts                       23.3      90.7
Trousers                     86.0      97.7
Flats                        73.8      97.6
Bra                          47.6      97.6
Dresses                      29.7      78.4
Sarees                        nan       nan
Earrings                     90.9     100.0
Deodorant                     0.0       0.0
Nail Polish                   0.0       0.0
Lipstick                      0.0       0.0
Track Pants                  53.5      88.4
Clutches                     36.4      90.9
Sweatshirts                  32.6      93.0
Caps                         82.5      82.5
Sweaters                     60.5      83.7
Ties                         85.3      88.2
Jackets                      44.2      81.4
Innerwear Vests               0.0       0.0
Kurtis                       48.6      97.1
Tunics                       19.2      92.3
Nightdress                   47.

## Compare this to not finetuning:

In [ ]:
data = fewshot.data.FashionData('../data/small/styles_quoted.csv',
                                '../data/small/images/',
                               train_transform=train_transform,
                               test_transform=test_transform,
                               top20=False)

In [ ]:
n_classes = len(data.idx2name)
n_classes

In [ ]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features

## replacing the last layer with the dense layer we need
model.fc = nn.Linear(num_ftrs, n_classes)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
fewshot.trainer.run_training(model, 5, data, optimizer, loss_func, gpu_id=0, root='./')